In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

from flipside import Flipside
flipside = Flipside("023b83a2-e411-4849-a961-3742c87bc9c9", "https://api-v2.flipsidecrypto.xyz")

We first set the symbol of the token we want to query data from.

In [2]:
token = "WETH"

This write the sql code and start query.

In [3]:
sql = """
SELECT HOUR, SYMBOL, PRICE
FROM ethereum.core.fact_hourly_token_prices
""" +\
"WHERE SYMBOL = '" + token + "'" 

query_result_set = flipside.query(sql, page_number = 1, page_size = 50000)

current_page_number = 1
page_size = 50000
total_pages = 2

# All the data will be stored in a list called "all_rows"
all_rows = []

while current_page_number <= total_pages:
  results = flipside.get_query_results(
    query_result_set.query_id,
    page_number=current_page_number,
    page_size=page_size
  )

  total_pages = results.page.totalPages
  if results.records:
      all_rows = all_rows + results.records
  
  current_page_number += 1

# Merge all rows to a dataframe
df = pd.DataFrame(all_rows)
df.drop(columns = ['__row_index'], inplace = True)
df['hour'] = pd.to_datetime(df['hour'])
df.sort_values(by = ['hour'], inplace = True)
df.head(10)

,hour,symbol,price
26513,2020-05-05 02:00:00+00:00,WETH,207.022268
26512,2020-05-05 03:00:00+00:00,WETH,207.983914
13872,2020-05-05 04:00:00+00:00,WETH,210.544355
22066,2020-05-05 05:00:00+00:00,WETH,209.690841
4767,2020-05-05 06:00:00+00:00,WETH,209.010651
9176,2020-05-05 07:00:00+00:00,WETH,209.010651
26514,2020-05-05 08:00:00+00:00,WETH,209.010651
8387,2020-05-05 09:00:00+00:00,WETH,209.533282
9373,2020-05-05 10:00:00+00:00,WETH,209.533282
13874,2020-05-05 11:00:00+00:00,WETH,209.533282


Lastly, we save the file.

In [4]:
df.to_csv("/Users/zhicong/Dropbox/DEXs/token_prices/" + token + ".csv", index=False)